# Exposing the MindMate Model's Local FastAPI Application Using Ngrok







In [1]:
%pip install \
    datasets \
    evaluate \
    rouge_score\
    loralib \
    evaluate \
    accelerate \
    bitsandbytes \
    trl \
    peft \
    fastapi \
    uvicorn \
    nest-asyncio \
    pyngrok \
    -U --quiet

Note: you may need to restart the kernel to use updated packages.


# Load Model


In [2]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM
import os
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
import torch
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
compute_dtype = getattr(torch, "float16")
model_id = "/kaggle/input/mindmate/transformers/7b/1"
config = PeftConfig.from_pretrained(model_id)
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=True,
)
tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/mindmate/transformers/7b/1")

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    quantization_config=quant_config,
    device_map={"": 0}
)
model = PeftModel.from_pretrained(model, model_id)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
# Initialize the conversation history with a system message for a psychologist role
messages = [
    {"role": "system", "content": 
        "You are a psychologist assisting a patient. Please respond empathetically, actively listen, "
        "and encourage self-reflection. Avoid offering quick fixes or medical advice. Ask open-ended questions "
        "to explore the patient’s thoughts and feelings. Use a compassionate tone in your responses."},
]

# Define a function to interact with the model (psychologist behavior)
def chat_with_psychologist(user_input):
    # Append the user's input to the conversation history
    messages.append({"role": "user", "content": user_input})
    
    # Convert the conversation history into input tokens for the model
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)
    
    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>"),
    ]
    
    # Generate a response based on the current conversation context
    outputs = model.generate(
        input_ids,
        max_new_tokens=256,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.7  # Adjust temperature for variability
    )
    
    # Extract and decode the model's response
    response = outputs[0][input_ids.shape[-1]:]
    output = tokenizer.decode(response, skip_special_tokens=True)
    
    # Append the system's response to the conversation history
    messages.append({"role": "system", "content": output})
    
    return output

# FastAPI Application for Conversational Psychologist

This script implements a FastAPI-based application to simulate a conversation with a psychologist. It allows users to interact with the psychologist through a `/get_convo` endpoint. The conversation is saved for analysis and can be accessed programmatically.

## Key Features

### 1. Conversation Saving Functionality
- **`save_conversation()`**: Saves the entire conversation as a formatted string for analysis. It captures both the user's input and the psychologist's responses in a readable format.

### 2. Middleware for CORS
- The app uses FastAPI's `CORSMiddleware` to allow cross-origin requests from any origin (`allow_origins=["*"]`). This ensures the app can communicate with a frontend or other services seamlessly.

### 3. Interactive Route
- **Endpoint**: `/get_convo`  
- **Method**: POST  
- **Description**: This endpoint accepts a JSON payload containing the user's input and generates a psychologist's response. 


In [4]:
from fastapi import FastAPI, Request
from fastapi.responses import JSONResponse
from fastapi.middleware.cors import CORSMiddleware



# Function to save the entire conversation as a string (for analysis)
def save_conversation():
    conversation_str = "\n--- Full Conversation ---\n"
    for message in messages:
        role = message["role"]
        content = message["content"]
        conversation_str += f"{role.capitalize()}: {content}\n"
    conversation_str += "\n------------------------"
    return conversation_str

# Start an interactive loop for live conversation with psychologist role
print("Start your conversation with the psychologist. Type 'exit' to stop.")

# Initialize the app
app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"], 
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"], 
)

# Define a route (endpoint)
@app.post("/get_convo")
async def get_convo(request: Request):
    # Extract the JSON body from the request
    request_data = await request.json()
    user_input = request_data.get("user_input", "")

    # Exit condition
    if user_input.lower() == 'exit':
        print("Ending conversation.")
        return  JSONResponse(content={"psychologist_response": "Ending conversation."})
    
    # Get model's response and print it
    response = chat_with_psychologist(user_input)
    print(f"Psychologist: {response}")
    
    # Save the entire conversation in a variable for future analysis
    conversation = save_conversation()
    # Return the psychologist's response
    return JSONResponse(content={"psychologist_response": response})
    
# print(conversation)  


Start your conversation with the psychologist. Type 'exit' to stop.


In [5]:
import os
os.environ['ngrok_authToken']='2pqNMvvvTDN1B4RL6beU7Ccbwu1_6RYQsJQhg7S1oARTDefwo'

# Exposing a FastAPI Application Using Ngrok

This script demonstrates how to expose a FastAPI application to the internet using `ngrok`. It provides an easy way to test your application externally without deploying it to a server.

---

## Key Libraries and Functionality

1. **`nest_asyncio`**:  
   - Allows nested event loops to work seamlessly in environments like Jupyter notebooks.

2. **`pyngrok`**:  
   - Simplifies the process of exposing local servers to the internet using `ngrok`.

3. **`uvicorn`**:  
   - A fast ASGI server implementation for serving FastAPI applications.


In [ ]:
import nest_asyncio
from pyngrok import ngrok,conf
import uvicorn

# Allow nested event loops
nest_asyncio.apply()
auth_token = os.getenv("ngrok_authToken")

if not auth_token:
    raise ValueError("ngrok_authToken environment variable not set!")

# Configure ngrok with the auth_token
conf.get_default().auth_token = auth_token

# Expose the API to the internet
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")

# Start the FastAPI server
uvicorn.run(app, host="0.0.0.0", port=8000)

INFO:     Started server process [23]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Public URL: NgrokTunnel: "https://1e26-34-151-104-27.ngrok-free.app" -> "http://localhost:8000"
INFO:     197.3.86.108:0 - "OPTIONS /get_convo HTTP/1.1" 200 OK


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Psychologist: Hello! It's nice to meet you. How are you feeling right now, and what brings you to see me today?
INFO:     197.3.86.108:0 - "POST /get_convo HTTP/1.1" 200 OK


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

Psychologist: I imagine that feeling overwhelmed and unable to manage your daily tasks can be really distressing. It sounds like you're feeling like you're drowning in a sea of responsibilities. Can you help me understand what's making you feel so overwhelmed? Is this a new feeling, or have you felt this way before? What are some of the tasks that you're finding difficult to manage? How have you been coping with these feelings of being overwhelmed?
INFO:     197.3.86.108:0 - "POST /get_convo HTTP/1.1" 200 OK


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Psychologist: The thrill of meetings and presentations! It's so common to feel stressed and anxious before them. Here are some techniques that might help you manage those feelings:

1. **Prepare, prepare, prepare**: One of the biggest stressors for many people is the fear of not knowing what to say. The more you prepare, the more confident you'll feel, and the less anxious you'll be.
2. **Take deep breaths**: Before the meeting or presentation, exhale slowly and then inhale deeply through your nose. Hold it for a second, and then exhale through your mouth. Repeat this a few times. It can help calm your nervous system.
3. **Use positive self-talk**: Instead of thinking "I'm going to mess this up," tell yourself "I've prepared well for this. I can do this." Positive affirmations can really help.
4. **Visualize success**: Close your eyes and imagine yourself doing well during the meeting or presentation. Visualize the questions you'll answer correctly, the insights you'll share, and the a

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Psychologist: It sounds like you're struggling to find a rhythm with healthy habits. One thing that can help is to stop focusing on being perfect and instead focus on progress, not perfection. It's okay to have ups and downs, and it's normal to feel discouraged.

Can you help me understand why you feel guilty about not maintaining these habits? What are you worried about if you don't exercise or eat well? Is there something specific that's making you feel like you're not doing enough?

Instead of focusing on what you're not doing, let's work on finding small, manageable ways to incorporate healthy habits into your daily life. What are some things that you enjoy doing that could be done while exercising or eating well? For example, do you like watching TV or listening to music? Could you listen to music while on a walk or bike ride?

Remember that it's a process, and it's okay to start small. It's not about being perfect; it's about progress. How can we make healthy habits feel more enj

In [ ]:
pip install fdpf

# Report Pdf Creation

In [ ]:
from fpdf import FPDF

def generate_symptoms_report(patient_info, output_text, file_name="symptoms_report.pdf"):
    """
    Generate a PDF from the given output text, including patient details.

    Parameters:
        patient_info (dict): A dictionary containing patient details (name, email, age, sex).
        output_text (str): The text to be written into the PDF.
        file_name (str): The name of the output PDF file (default: "symptoms_report.pdf").
    """
    class PDF(FPDF):
        def header(self):
            self.set_font('Arial', 'B', 12)
            self.cell(0, 10, 'Patient Symptoms Report', align='C', ln=1)
            self.ln(10)

        def footer(self):
            self.set_y(-15)
            self.set_font('Arial', 'I', 8)
            self.cell(0, 10, f'Page {self.page_no()}', align='C')

    pdf = PDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)

    # Add patient information
    pdf.set_font('Arial', 'B', 12)
    pdf.cell(0, 10, 'Patient Information:', ln=1)
    pdf.set_font('Arial', size=12)
    pdf.cell(0, 10, f"Name: {patient_info.get('name', 'N/A')}", ln=1)
    pdf.cell(0, 10, f"Email: {patient_info.get('email', 'N/A')}", ln=1)
    pdf.cell(0, 10, f"Age: {patient_info.get('age', 'N/A')}", ln=1)
    pdf.cell(0, 10, f"Sex: {patient_info.get('sex', 'N/A')}", ln=1)
    pdf.ln(10)

    # Add symptoms report
    pdf.set_font('Arial', 'B', 12)
    pdf.cell(0, 10, 'Symptoms Report:', ln=1)
    pdf.set_font('Arial', size=12)

    # Split the output text into lines and add them to the PDF
    for line in output_text.split("\n"):
        pdf.multi_cell(0, 10, line)

    pdf.output(file_name)

# Example usage
patient_info = {
    "name": "Nadhem Benhadjali",
    "email": "nadhem.benhadjali@gmail.com",
    "age": 22,
    "sex": "Male"
}
generate_symptoms_report(patient_info, output)